In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 12.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.1/124.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#hide
from fastbook import *

# Training a State-of-the-Art Model

## Imagenette

In [3]:
from fastai.vision.all import *
path = untar_data(URLs.IMAGENETTE)

In [4]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = dblock.dataloaders(path, bs=64)

# Datablock semelhante ao abordado em cadernos anteriores

In [5]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

# xresnet50: é uma função que retorna uma rede neural com arquitetura pré-definida, mas sem pré treinamento.
# n_out=dls.c: define o número de saídas (classes) do modelo.
# dls.c pega automaticamente quantas categorias diferentes existem no conjunto de dados (por exemplo, 10 se for classificação de 10 tipos de imagens).

epoch,train_loss,valid_loss,accuracy,time
0,1.590252,1.672649,0.463032,02:24
1,1.197253,1.363934,0.560866,02:32
2,0.961066,1.037857,0.661688,02:32
3,0.734662,0.658592,0.794249,02:33
4,0.609249,0.545648,0.830097,02:32


## Normalization

In [6]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

# "Para cada canal de cor, calcule a média e dp de todos os pixels em todas as imagens."
# Ele pega um batch (lote) de dados do DataLoader.
# x: são as imagens (normalmente um tensor com shape [bs, 3, h, w] = lote, canais, altura, largura)

# O que faz x.mean(dim=[0,2,3])?
# Isso calcula a média dos pixels das imagens:
# dim=0: percorre todos os itens do batch
# dim=2,3: percorre altura e largura da imagem (pixels)
# Deixa de fora dim=1, que representa os 3 canais de cor: vermelho, verde e azul (RGB).

# Resultado: você obtém uma média por canal de cor: [Média dos valores do canal vermelho, Média dos valores do canal verde, Média dos valores do canal azul]

(TensorImage([0.4675, 0.4609, 0.4204], device='cuda:0'),
 TensorImage([0.2911, 0.2801, 0.2950], device='cuda:0'))

In [8]:
def get_dls(bs, size):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
                               Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(path, bs=bs)


# Esse datablock vira uma função reutilizável, que recebe:
# bs: tamanho do batch
# size: tamanho da imagem final usada no treinamento (ex: 224)
# Isso te permite testar diferentes tamanhos sem reescrever o DataBlock.

# Normalize.from_stats(...)
# Normaliza canal por canal (R, G, B), usando as médias e desvios padrão do ImageNet
# *aug_transforms(...)	Desempacota a lista de transformações para combiná-las com Normalize

# Importante: Quando você usa vision_learner, o fastai já aplica normalização automaticamente, porque ele sabe que você está usando um modelo pré-treinado (como ResNet ou XResNet).

In [9]:
dls = get_dls(64, 224)

In [10]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([-0.0620,  0.0279,  0.1135], device='cuda:0'),
 TensorImage([1.1985, 1.2326, 1.3172], device='cuda:0'))

In [11]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.633674,2.805343,0.398058,02:35
1,1.289020,2.187535,0.471621,02:33
2,0.955947,1.016960,0.674384,02:33
3,0.740758,0.687258,0.787528,02:33
4,0.612378,0.575446,0.815534,02:33


## Progressive Resizing

In [12]:
dls = get_dls(128, 128)
learn = Learner(dls, xresnet50(n_out=dls.c), loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy)
learn.fit_one_cycle(4, 3e-3)

# dls = get_dls(128, 128) Você está criando um DataLoaders com:
# Batch size = 128
# Tamanho das imagens = 128×128
# Ou seja, as imagens são pequenas no começo → mais rápido de treinar.
# Treina por 4 épocas com imagens pequenas.

# Treinar imagens pequenas é mais rápido
# O modelo pode aprender formas básicas e padrões globais
# Menos custo computacional
# Serve como um pré-treinamento leve
# Melhora generalização - é uma especie de data augmentation

epoch,train_loss,valid_loss,accuracy,time
0,1.616036,3.027645,0.391337,01:17
1,1.234733,2.027993,0.530246,01:17
2,0.957411,0.985928,0.702390,01:17
3,0.730644,0.674533,0.794249,01:16


In [13]:
learn.dls = get_dls(64, 224)
learn.fine_tune(5, 1e-3)

# Você substitui os dados do Learner por um novo DataLoaders, agora com:
# Batch size = 64 (menor, pois imagem maior consome mais memória)
# Imagem maior = 224×224
# Isso é o resize progressivo: agora o modelo vai treinar com mais detalhes visuais, pois as imagens têm mais resolução.
# E como o modelo já aprendeu coisas úteis nas imagens menores, ele agora refina o que sabe com mais detalhes.

epoch,train_loss,valid_loss,accuracy,time
0,0.819846,1.182295,0.664302,02:33


epoch,train_loss,valid_loss,accuracy,time
0,0.643167,0.800408,0.765497,02:31
1,0.678964,0.772617,0.757655,02:32
2,0.598801,0.522853,0.843167,02:32
3,0.489412,0.469953,0.859597,02:32
4,0.434970,0.439171,0.864824,02:31


## Test Time Augmentation

In [15]:
preds,targs = learn.tta()
accuracy(preds, targs).item()

# O que learn.tta() faz?
# Aplica várias transformações leves (como flip horizontal, zoom, rotação leve, etc.) a cada imagem do conjunto de validação.
# Faz várias predições por imagem (uma para cada versão transformada).
# Depois, tira a média das predições de cada imagem.
# Retorna:
# preds: tensor com as predições médias finais para cada imagem. 
# targs: rótulos verdadeiros do conjunto de validação.

# Agora, o que accuracy(preds, targs) faz:
# preds: um tensor com médias das predições softmax para cada imagem ([n_imagens, n_classes])
# targs: os rótulos verdadeiros, como inteiros ([n_imagens])

# Para cada linha de preds, pega o índice da classe com maior valor, compara com os rótulos verdadeiros (targs), calcula a proporção de acertos.

0.869305431842804

## Mixup

### Sidebar: Papers and Math

### End sidebar

In [ ]:
church = PILImage.create(get_image_files_sorted(path/'train'/'n03028079')[0])
gas = PILImage.create(get_image_files_sorted(path/'train'/'n03425413')[0])
church = church.resize((256,256))
gas = gas.resize((256,256))
tchurch = tensor(church).float() / 255.
tgas = tensor(gas).float() / 255.

_,axs = plt.subplots(1, 3, figsize=(12,4))
show_image(tchurch, ax=axs[0]);
show_image(tgas, ax=axs[1]);
show_image((0.3*tchurch + 0.7*tgas), ax=axs[2]);

# Pegando uma imagem de uma igreja (church) e uma de um posto de gasolina (gas)
# Redimensionando as duas para 256×256
# Convertendo para tensores e normalizando os valores de pixel para o intervalo [0,1]
# Mostrando:
# A imagem da igreja
# A imagem do posto
# Uma interpolação das duas (com pesos 0.3 e 0.7)

In [ ]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy, cbs=MixUp())
learn.fit_one_cycle(5, 3e-3)

# ✅ cbs=MixUp()
# Isso ativa o callback MixUp
# Durante o treinamento, ele pega pares aleatórios de imagens e rótulos, e faz uma interpolação aleatória entre eles
# O peso de mistura (ex: 0.3/0.7) é escolhido de uma distribuição Beta (por padrão, Beta(0.4, 0.4))

# O que acontece nos bastidores:
# As imagens são misturadas: x = λ * x1 + (1 - λ) * x2
# Os rótulos são misturados: y = λ * y1 + (1 - λ) * y2
# O modelo é treinado para prever a distribuição entre classes
# CrossEntropyLossFlat lida automaticamente com esses rótulos mistos

## Label Smoothing

In [16]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

# Você está criando um modelo com a função de perda LabelSmoothingCrossEntropy, em vez da tradicional CrossEntropyLossFlat.

# O que é Label Smoothing?
# É uma técnica onde você não usa rótulos “duros” (one-hot 100%), e sim distribuições suavizadas.
# Em vez de representar a classe correta como [0, 0, 1, 0, 0]
# Você representa como [0.01, 0.01, 0.96, 0.01, 0.01] (valores aproximados)

# Por que isso ajuda?
# Evita que o modelo fique “super confiante” (probabilidade de 100% em uma só classe)
# Ajuda na regularização (parecido com dropout)
# Reduz overfitting
# Pode melhorar a calibração das probabilidades.

# Importante: o benefício dessa técnica só aparece quando o modelo é treinado por muitas épocas. No começo o desempenho pode ser pior

# Por que precisa de mais épocas para funcionar bem?
# 1. Porque ela freia o aprendizado "rápido e fácil"
# Sem label smoothing, o modelo pode decorar rapidamente os rótulos, especialmente em datasets pequenos ou fáceis
# Com smoothing, ele precisa pensar mais — o aprendizado inicial é um pouco mais lento, mas tende a ser mais robusto a longo prazo

# 2. Porque o modelo precisa tempo para se adaptar
# A perda suavizada não dá uma recompensa tão “clara” a cada acerto
# O modelo leva mais tempo para identificar padrões úteis reais, já que não é punido ou premiado de forma tão extrema

# 3. Porque os benefícios aparecem na generalização, não no overfitting rápido
# Se você treinar por poucas épocas, pode parecer que o modelo com label smoothing está “pior”
# Mas com mais treino, ele tende a ter melhor desempenho no conjunto de validação/teste

# O benefício real pode so aparecer depois de 10–20 épocas (dataset pequeno), 15–30 épocas (dataset médio), 30–90+ épocas (dataset grande)

epoch,train_loss,valid_loss,accuracy,time
0,1.766246,2.423440,0.457804,01:15
1,1.525698,1.377476,0.643764,01:16
2,1.306419,1.261292,0.690067,01:16
3,1.142434,1.228609,0.706497,01:17
4,1.035957,0.996405,0.806946,01:17


### Sidebar: Label Smoothing, the Paper

### End sidebar

## Conclusion

## Questionnaire

1. What is the difference between ImageNet and Imagenette? When is it better to experiment on one versus the other?
1. What is normalization?
1. Why didn't we have to care about normalization when using a pretrained model?
1. What is progressive resizing?
1. Implement progressive resizing in your own project. Did it help?
1. What is test time augmentation? How do you use it in fastai?
1. Is using TTA at inference slower or faster than regular inference? Why?
1. What is Mixup? How do you use it in fastai?
1. Why does Mixup prevent the model from being too confident?
1. Why does training with Mixup for five epochs end up worse than training without Mixup?
1. What is the idea behind label smoothing?
1. What problems in your data can label smoothing help with?
1. When using label smoothing with five categories, what is the target associated with the index 1?
1. What is the first step to take when you want to prototype quick experiments on a new dataset?

### Further Research

1. Use the fastai documentation to build a function that crops an image to a square in each of the four corners, then implement a TTA method that averages the predictions on a center crop and those four crops. Did it help? Is it better than the TTA method of fastai?
1. Find the Mixup paper on arXiv and read it. Pick one or two more recent articles introducing variants of Mixup and read them, then try to implement them on your problem.
1. Find the script training Imagenette using Mixup and use it as an example to build a script for a long training on your own project. Execute it and see if it helps.
1. Read the sidebar "Label Smoothing, the Paper", look at the relevant section of the original paper and see if you can follow it. Don't be afraid to ask for help!